In [20]:
import gym
import numpy as np
import random
from IPython.display import clear_output
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# https://towardsdatascience.com/reinforcement-learning-with-python-8ef0242a2fa2
# Init Taxi-V2 Env
env = gym.make("Taxi-v3").env

# Init arbitary values
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.7 # Momemtum 0.2, Current 0.8 Greedy, 0.2 is to reduce volatality and flip flop
gamma = 0.2 # Learning Rate 0.1 Greedyness is 10%
epsilon = 0.1 # explore 10% exploit 90%


all_epochs = []
all_penalties = []
training_memory = []

for i in range(1, 50000):
    state = env.reset()

    # Init Vars
    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            # Check the action space
            action = env.action_space.sample()
        else:
            # Check the learned values
            action = np.argmax(q_table[state])

        next_state, reward, done, info = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        # Update the new value
        new_value = (1 - alpha) * old_value + alpha * \
            (reward + gamma * next_max)
        q_table[state, action] = new_value        
        
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
    

    if i % 100 == 0:
        training_memory.append(q_table.copy())
        clear_output(wait=True)
        print("Episode:", i)
        print("Saved q_table during training:", i)

print("Training finished.")
print(q_table)

Episode: 49900
Saved q_table during training: 49900
Training finished.
[[  0.           0.           0.           0.           0.
    0.        ]
 [ -1.24999956  -1.24999782  -1.24999956  -1.24999782  -1.24998912
  -10.24999782]
 [ -1.249728    -1.24864     -1.249728    -1.24864     -1.2432
  -10.24864   ]
 ...
 [ -1.24266508  -1.216       -1.24251888  -1.24818805  -7.
  -10.21665611]
 [ -1.24997104  -1.2499456   -1.24998367  -1.2499456   -7.
  -10.2420912 ]
 [ -0.40037179  -1.0584      -0.5824       3.          -9.3352
   -9.39997766]]


** There are four designated locations in the grid world indicated by R(ed), B(lue), G(reen), and Y(ellow). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drive to the passenger's location, pick up the passenger, drive to the passenger's destination (another one of the four specified locations), and then drop off the passenger. Once the passenger is dropped off, the episode ends. There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger (including the case when the passenger is the taxi), and 4 destination locations. Actions: There are 6 discrete deterministic actions: **

    0: move south
    1: move north
    2: move east
    3: move west
    4: pickup passenger
    5: dropoff passenger
Rewards: There is a reward of -1 for each action and an additional reward of +20 for delievering the passenger. There is a reward of -10 for executing actions "pickup" and "dropoff" illegally. Rendering:

    blue: passenger
    magenta: destination
    yellow: empty taxi
    green: full taxi
    other letters: locations


state space is represented by:
    (taxi_row, taxi_col, passenger_location, destination)

In [2]:
# At state 499 i will definitely move west
state = 499
print(training_memory[0][state])
print(training_memory[20][state])
print(training_memory[50][state])
print(training_memory[200][state])

[-0.196      -0.19       -0.196       0.69808456 -1.         -1.        ]
[-0.196      -0.19       -0.196       6.56515184 -1.         -1.85811493]
[-0.196      -0.19       -0.196      10.08690409 -1.         -1.85811493]
[ 0.36812688  0.28922308 -0.196      10.99948509 -1.47548993 -1.85811493]


In [3]:
# At state 77 i will definitely move east
state = 77
print(training_memory[0][state])
print(training_memory[20][state])
print(training_memory[50][state])
print(training_memory[200][state])

[-0.20176    -0.19        1.56090623 -0.20176    -1.         -1.        ]
[-0.20176    -0.19        8.78744551 -0.3025382  -1.         -1.66257716]
[-0.20176    -0.19       10.66790744 -0.3025382  -1.         -1.93309658]
[-0.20176     0.38844581 10.99951661 -0.35312314 -1.2561397  -2.08054712]


In [4]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [8]:
action_dict = {0:  "move south"
,1: "move north"
,2: "move east"
,3: "move west"
,4: "pickup passenger"
,5: "dropoff passenger"
}

ENV_STATE = env.reset()
print(env.render(mode='ansi'))
state_memory = [i[ENV_STATE] for i in training_memory]
printmd("For state **{}**".format(ENV_STATE))
for step, i in enumerate(state_memory):
    
    if step % 20==0:
        choice = np.argmax(i)
        printmd("for episode in {}, q table action is {} and it will ... **{}**".format(step*100, choice, action_dict[choice]))
        print(i)
        print()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




For state **92**

for episode in 0, q table action is 0 and it will ... **move south**

[-1.24970578 -1.24974593 -1.24985543 -1.24981037 -6.         -6.        ]



for episode in 2000, q table action is 3 and it will ... **move west**

...

[ -1.24999998  -1.24999999  -1.24999998  -1.24999998  -9.978
 -10.14119972]



for episode in 46000, q table action is 0 and it will ... **move south**

[ -1.24999998  -1.24999999  -1.24999998  -1.24999998  -9.978
 -10.14119972]



for episode in 48000, q table action is 0 and it will ... **move south**

[ -1.24999998  -1.24999999  -1.24999999  -1.24999998 -10.1412
 -10.14119972]



In [17]:
import time
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Episode: {frame['episode']}")
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        time.sleep(0.8)

total_epochs, total_penalties = 0, 0
episodes = 10
frames = []

for ep in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        env
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1
        
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'episode': ep, 
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print_frames(frames)

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode: 9
Timestep: 132
State: 0
Action: 5
Reward: 20
Results after 10 episodes:
Average timesteps per episode: 13.2
Average penalties per episode: 0.0
